# Local LLM Metrics: Latency, Throughput, and Parameters

In the previous notebooks, we explored metrics for Classification and Regression. In the world of Generative AI and Large Language Models (LLMs), accuracy is harder to quantify mathematically. 

Instead, AI Project Managers must focus on **Operational Metrics** and **Behavioral Parameters**.

## Goals
In this notebook, you will not just run a model; you will measure its performance characteristics.
1. **Connect** to your local Ollama instance via Python.
2. **Measure Latency:** How long does the user wait?
3. **Measure Throughput:** How fast is text generated (Tokens per Second)?
4. **Experiment with Temperature:** How does randomness affect output?

### Prerequisites
Ensure Ollama is running in your terminal (`ollama serve` or the desktop app) and you have pulled a model (e.g., `ollama pull llama3.2:3b`).

## 1. The API Connection

Unlike the previous notebooks where we used `scikit-learn` to run calculations internally, LLMs usually run as a separate **Service**. We communicate with them using HTTP requests (APIs).

In [ ]:
import requests
import json
import time

# Configuration
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:3b"  # Change if you want to use a different model

def check_server_status():
    try:
        # Simple GET request to see if the server is up
        response = requests.get("http://localhost:11434/")
        if response.status_code == 200:
            print("✅ Ollama Server is running!")
        else:
            print(f"⚠️ Server returned status: {response.status_code}")
    except requests.exceptions.ConnectionError:
        print("❌ Could not connect to Ollama. Is the app running?")

check_server_status()

In [ ]:
# Create a persistent session to reuse connections for all requests
session = requests.Session()

## 2. Measuring Latency and Throughput

For a PM, "Speed" means two different things:

1. **Latency (Time to First Token):** How long before the AI *starts* writing? (Crucial for Chatbots).
2. **Throughput (Tokens per Second):** How fast does it write the whole answer? (Crucial for summarizing large documents).

Let's write a function to measure this. We will send a `POST` request to the API.

In [ ]:
import time
import requests

def query_model_with_metrics(prompt, model=MODEL_NAME):
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    # Default values (safe fallbacks)
    text = ""
    token_count = 0
    error = None
    
    # Start Timer
    start_time = time.perf_counter()

    try:
        # CHANGE HERE: Use 'session.post' instead of 'requests.post'
        response = session.post(OLLAMA_URL, json=payload)
        response.raise_for_status()  
        
        # Parse logic
        data = response.json()
        text = data.get("response", "")
        token_count = data.get("eval_count", 0)

    except requests.exceptions.RequestException as e:
        error = f"Network/API Error: {e}"
    except ValueError:
        error = "Invalid JSON received from server"
    except Exception as e:
        error = f"Unexpected Error: {e}"

    # Stop Timer
    duration = time.perf_counter() - start_time
    
    # Calculate throughput (avoid division by zero)
    tps = token_count / duration if duration > 0 else 0.0

    return {
        "text": text,
        "duration_seconds": duration,
        "token_count": token_count,
        "tokens_per_sec": tps,
        "error": error
    }

# Test it out
metrics = query_model_with_metrics("Explain Quantum Computing to a 5 year old.")

if metrics.get("error"):
    print(f"Error: {metrics['error']}")
else:
    print(f"Response: {metrics['text'][:200]}...")
    print(f"\n⏱️ Total Duration: {metrics['duration_seconds']:.2f}s")
    print(f"🚀 Speed: {metrics['tokens_per_sec']:.2f} tokens/sec")

### Reflection
Look at your `tokens/sec`. 
* **Human reading speed:** ~3-5 tokens/second.
* **Typical GPU speed:** >50 tokens/second.
* **Typical CPU (Laptop) speed:** 5-20 tokens/second.

If your local number is lower than human reading speed, the User Experience (UX) will feel "laggy."

## 3. The Temperature Parameter

In the `Classification Metrics` notebook, we discussed thresholds (0.5 vs 0.25) changing the outcome. In LLMs, the equivalent lever is **Temperature**.

* **Temperature 0.0:** Deterministic. Always picks the most likely next word. Good for coding, data extraction.
* **Temperature 0.8+:** Creative. Picks less likely words. Good for poetry, brainstorming.

In [ ]:
def query_with_temp(prompt, temperature):
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": temperature}
    }
    try:
        response = session.post(OLLAMA_URL, json=payload)
        response.raise_for_status() # Raises error for 4xx/5xx codes
        return response.json().get("response", "")
    except Exception as e:
        return f"Error: {str(e)}"

test_prompt = "Propose 3 unique names for a new coffee shop in Berlin."
print(query_with_temp(test_prompt, 0.0))
print(query_with_temp(test_prompt, 0.9))

## 4. System Prompts (The Persona)

Just as we set up requirements before coding, we set up **System Prompts** before the user interacts with the AI. This guides the model's behavior throughout the conversation.

In [ ]:
def query_with_system_prompt(user_input, system_role):
    payload = {
        "model": MODEL_NAME,
        "prompt": user_input,
        "system": system_role,
        "stream": False
    }
    
    try:
        response = session.post(OLLAMA_URL, json=payload)
        response.raise_for_status() # Catches 404s/500s
        return response.json().get("response", "")
    except Exception as e:
        return f"Error: {str(e)}"

user_q = "I made a mistake in production."

# Scenario A: The Empathetic Manager
role_a = "You are an empathetic career coach. Your tone is soothing and supportive."
print(f"\n[Coach]: {query_with_system_prompt(user_q, role_a)}")

# Scenario B: The Strict Technical Lead
role_b = "You are a strict Senior DevOps Engineer. You focus only on root cause analysis and immediate fixes. Be brief."
print(f"\n[DevOps]: {query_with_system_prompt(user_q, role_b)}")

## Summary

You have now successfully:
1. Built a basic **LLM Client** in Python.
2. Calculated **Throughput** (a key cost/performance driver).
3. Manipulated **Temperature** to change creativity.
4. Used **System Prompts** to engineer specific behaviors.

### Next Step
Try integrating this script into a loop to build a CLI Chatbot!